# Part 4: Disaster Strikes

In [1]:
#Import statements
import os
import requests
import json
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#q8
! hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 15784800256 (14.70 GB)
DFS Remaining: 15526035456 (14.46 GB)
DFS Used: 258764800 (246.78 MB)
DFS Used%: 1.64%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 90
	Missing blocks (with replication factor 1): 90
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.24.0.5:9866 (project-4-p4-rfwong07-dn-1.project-4-p4-rfwong07_default)
Hostname: 661631f2ec8b
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 258764800 (246.78 MB)
Non DFS Used: 10019475456 (9.33 GB)
DFS Remaining: 15526035456 (14.46 GB)
DFS Used%: 1.00

In [3]:
#q9
namenode_url = "http://boss:9870/webhdfs/v1/single.csv"
location_url = f"{namenode_url}?op=GETFILEBLOCKLOCATIONS" #Reference https://hadoop.apache.org/docs/r3.3.6/hadoop-project-dist/hadoop-hdfs/WebHDFS.html#Get_File_Block_Locations
r = requests.get(location_url)
block_location_dict = r.json()
block_location_list = block_location_dict['BlockLocations']['BlockLocation']
hosts_count_dict = {} #Initiate counting dictionary
for item in block_location_list: #This loops through each item in the list and count up the "hosts" value
    cntr_value = item['hosts']
    if cntr_value == []: #When the list is empty up the count in the lost key
        if 'lost' not in hosts_count_dict:
            hosts_count_dict['lost'] = 1
        else:
            hosts_count_dict['lost'] += 1
    elif cntr_value[0] not in hosts_count_dict:
        hosts_count_dict[cntr_value[0]] = 1

    else:
        hosts_count_dict[cntr_value[0]] += 1
hosts_count_dict

{'lost': 90, '661631f2ec8b': 77}

In [4]:
#This block of code is to find all the indexes (blocks) that will have a host or not.
#Then separates them into their own list of indexes
namenode_url = "http://boss:9870/webhdfs/v1/single.csv"
location_url = f"{namenode_url}?op=GETFILEBLOCKLOCATIONS"
r = requests.get(location_url)
block_location_list = r.json()['BlockLocations']['BlockLocation']
len_block_location_list = len(block_location_list)

#Empty list to keep track of indexes (blocks)
lost_list = []
host_list = []

#For loop for separating the indexes that will raise status code 403 or 200
for idx in range(len_block_location_list):
    cntr_val = block_location_list[idx]['hosts']
    if cntr_val == []:
        lost_list.append(idx)
    else:
        host_list.append(idx)



In [5]:
#q10
namenode_url = "http://boss:9870/webhdfs/v1/single.csv"
block_size = 1048576
single_family_count = 0 # Counter for "Single Family" string
for host_idx in host_list:
    location_url = f"{namenode_url}?op=OPEN&offset={block_size * host_idx}&noredirect=true" #Create the url with ?op=GETFILESTATUS
    r = requests.get(location_url)
    #Condtional for Status Code == 200 though all indexes in host_list should be 200
    #Because host_list created in last cell should only contain non 403 blocks.
    if r.status_code == 200:
        #print(r.json()['Location'])
        hdfs = pa.fs.HadoopFileSystem("boss", 9000)
        with hdfs.open_input_file('/single.csv') as f:
            string = str(f.read_at(block_size, block_size * host_idx), 'utf-8')
            string_appear_count = string.count("Single Family")
            single_family_count += string_appear_count
    else:
        continue

single_family_count

2024-03-17 16:23:01,146 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


205313